# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
#from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download(['words', 'punkt', 'stopwords',
               'averaged_perceptron_tagger',
               'maxent_ne_chunker', 'wordnet'])

[nltk_data] Downloading package words to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('message_category', engine).sample(8000)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
19416,22660,A joint UNEP/OCHA Environmental Risk Identific...,None,news,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,1,0
22617,26205,They are one of fourteen families living in la...,None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.nunique()

id                        7994
message                   7994
original                  3062
genre                        3
related                      3
request                      2
offer                        2
aid_related                  2
medical_help                 2
medical_products             2
search_and_rescue            2
security                     2
military                     2
child_alone                  1
water                        2
food                         2
shelter                      2
clothing                     2
money                        2
missing_people               2
refugees                     2
death                        2
other_aid                    2
infrastructure_related       2
transport                    2
buildings                    2
electricity                  2
tools                        2
hospitals                    2
shops                        2
aid_centers                  2
other_infrastructure         2
weather_

In [5]:
df.count()

id                        8000
message                   8000
original                  3106
genre                     8000
related                   8000
request                   8000
offer                     8000
aid_related               8000
medical_help              8000
medical_products          8000
search_and_rescue         8000
security                  8000
military                  8000
child_alone               8000
water                     8000
food                      8000
shelter                   8000
clothing                  8000
money                     8000
missing_people            8000
refugees                  8000
death                     8000
other_aid                 8000
infrastructure_related    8000
transport                 8000
buildings                 8000
electricity               8000
tools                     8000
hospitals                 8000
shops                     8000
aid_centers               8000
other_infrastructure      8000
weather_

In [6]:
# drop id, original
df.drop(['id', 'original'], axis=1, inplace=True)

In [7]:
# check unique values for 'related'
df['related'].unique()

array([1, 0, 2])

In [8]:
# number of 'related' values that are 2
(df['related']==2).sum()

52

In [9]:
# replace 'related' values 2's with 1's
df['related'].replace(2,1, inplace=True)

In [10]:
df.head(2)

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
19416,A joint UNEP/OCHA Environmental Risk Identific...,news,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
22617,They are one of fourteen families living in la...,news,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = df['message'].values
Y = df.loc[:,'related':'direct_report'].values

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # replace urls with placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # convert to lowercase and remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    # remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    # reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    # lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]

    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_features=None, max_df=1.0)),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(
        max_depth=None, min_samples_split=2, max_leaf_nodes=None,
        min_samples_leaf=1, max_samples=None, n_estimators=100))),
])

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a24d0af80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

In [16]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
Y_pred = pipeline.predict(X_test)

In [18]:
(Y_pred == Y_test).mean()

0.9450694444444444

In [19]:
def display_results(y_test, y_pred, y_col):
    
    clf_report = classification_report(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    accuracy = (y_pred == y_test).mean() 
    print('\n')   
    print(y_col, ":")
    print('\n')  
    print(clf_report)
    print(confusion_mat)
    print('\n')
    print('Accuracy =', accuracy)
    print('-'*65)

In [20]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:,i], Y_pred[:,i], df.loc[:,'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.69      0.34      0.46       572
           1       0.82      0.95      0.88      1828

    accuracy                           0.81      2400
   macro avg       0.75      0.65      0.67      2400
weighted avg       0.79      0.81      0.78      2400

[[ 196  376]
 [  89 1739]]


Accuracy = 0.80625
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1996
           1       0.82      0.46      0.59       404

    accuracy                           0.89      2400
   macro avg       0.86      0.72      0.76      2400
weighted avg       0.89      0.89      0.88      2400

[[1956   40]
 [ 219  185]]


Accuracy = 0.8920833333333333
-----------------------------------------------------------------


offer :


              precision    recall  f1-score   support

           0

/Users/ayemyatwinshwe/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




tools :


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2381
           1       0.00      0.00      0.00        19

    accuracy                           0.99      2400
   macro avg       0.50      0.50      0.50      2400
weighted avg       0.98      0.99      0.99      2400

[[2381    0]
 [  19    0]]


Accuracy = 0.9920833333333333
-----------------------------------------------------------------


hospitals :


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2368
           1       0.00      0.00      0.00        32

    accuracy                           0.99      2400
   macro avg       0.49      0.50      0.50      2400
weighted avg       0.97      0.99      0.98      2400

[[2368    0]
 [  32    0]]


Accuracy = 0.9866666666666667
-----------------------------------------------------------------


shops :


              precision    recall  f1-score   support

 

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
%timeit

parameters = {

    # 'vect__max_df': [0.75, 1.0], # 1.0
    # 'vect__max_features': [None, 5000, 10000], # None

    # 'tfidf__use_idf': [True, False], # True

    # 'clf__estimator__max_depth': [2, 50, None], # None
    # 'clf__estimator__min_samples_split': [2, 100],
    # 'clf__estimator__max_leaf_nodes': [16, None],
    # 'clf__estimator__min_samples_leaf': [1, 16],
    'clf__estimator__max_samples': [None, 0.75],  # None
    'clf__estimator__n_estimators': [50, 100, 150]  # 100

}

cv = GridSearchCV(pipeline, param_grid=parameters,
                  scoring='f1_micro', cv=2, n_jobs=-1, verbose=2)

cv.fit(X_train, Y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  1.1min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.2min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
Y_pred_grid = cv.predict(X_test)

In [23]:
(Y_pred_grid == Y_test).mean()

0.945162037037037

In [24]:
cv.get_params()

{'cv': 2,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a24d0af80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                      

In [25]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:,i], Y_pred_grid[:,i], df.loc[:,'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.68      0.33      0.45       572
           1       0.82      0.95      0.88      1828

    accuracy                           0.80      2400
   macro avg       0.75      0.64      0.66      2400
weighted avg       0.79      0.80      0.78      2400

[[ 191  381]
 [  89 1739]]


Accuracy = 0.8041666666666667
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1996
           1       0.82      0.46      0.59       404

    accuracy                           0.89      2400
   macro avg       0.86      0.72      0.76      2400
weighted avg       0.89      0.89      0.88      2400

[[1956   40]
 [ 220  184]]


Accuracy = 0.8916666666666667
-----------------------------------------------------------------


offer :


              precision    recall  f1-score   support

 

/Users/ayemyatwinshwe/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




hospitals :


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2368
           1       0.00      0.00      0.00        32

    accuracy                           0.99      2400
   macro avg       0.49      0.50      0.50      2400
weighted avg       0.97      0.99      0.98      2400

[[2368    0]
 [  32    0]]


Accuracy = 0.9866666666666667
-----------------------------------------------------------------


shops :


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2389
           1       0.00      0.00      0.00        11

    accuracy                           1.00      2400
   macro avg       0.50      0.50      0.50      2400
weighted avg       0.99      1.00      0.99      2400

[[2389    0]
 [  11    0]]


Accuracy = 0.9954166666666666
-----------------------------------------------------------------


aid_centers :


              precision    recall  f1-score   supp

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.